In [ ]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import sqlite3
import csv
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
#command to install: conda install -c conda-forge wordcloud
from wordcloud import WordCloud
import re
import os
from sqlalchemy import create_engine # database connection
import datetime as dt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
from sklearn.metrics import f1_score,precision_score,recall_score
from sklearn import svm
from sklearn.linear_model import LogisticRegression
#from skmultilearn.adapt import mlknn
#from skmultilearn.problem_transform import ClassifierChain
#from skmultilearn.problem_transform import BinaryRelevance
#from skmultilearn.problem_transform import LabelPowerset
from sklearn.naive_bayes import GaussianNB
from datetime import datetime

<h3> 3.3 Cleaning and preprocessing of Questions </h3>

<h3> 3.3.1 Preprocessing </h3>

<ol> 
    <li> Sample 1M data points </li>
    <li> Separate out code-snippets from Body </li>
    <li> Remove Spcial characters from Question title and description (not in code)</li>
    <li> Remove stop words (Except 'C') </li>
    <li> Remove HTML Tags </li>
    <li> Convert all the characters into small letters </li>
    <li> Use SnowballStemmer to stem the words </li>
</ol>

In [ ]:
def striphtml(data):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', str(data))
    return cleantext
stop_words = set(stopwords.words('english'))
stemmer = SnowballStemmer("english")

In [ ]:
#http://www.sqlitetutorial.net/sqlite-python/create-tables/
#create a connection object using connect() function of the sqlite module
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
        
    return None

#create a cursor object by calling the cursor method of connection object
#pass the create table sql statement to the execute() method of cursor object and it executes the statement
def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)
        

#check if the created table exists in the database
def checkTableExists(dbcon):
    cursr = dbcon.cursor()
    str = "select name from sqlite_master where type='table'"
    table_names = cursr.execute(str)
    print("Tables in the database are: ")
    tables = table_names.fetchall()
    print(tables[0][0])
    return(len(tables))

#create database and table in it
def create_database_table(database, query):
    conn = create_connection(database)
    if conn is not None:
        create_table(conn, query)
        checkTableExists(conn)
    else:
        print("Error! Cannot create the database connection.")
    conn.close()
    
sql_create_table = """CREATE TABLE IF NOT EXISTS QuestionsProcessed (question text NOT NULL, code text, tags text, words_pre integer, words_post integer, is_code integer);"""
create_database_table("Processed.db", sql_create_table)


In [ ]:
# http://www.sqlitetutorial.net/sqlite-delete/
# https://stackoverflow.com/questions/2279706/select-random-row-from-a-sqlite-table
start = dt.datetime.now()
read_db = "train_no_dup.db"
write_db = "Processed.db"

if os.path.isfile(read_db):
    conn_r = create_connection(read_db)
    if conn_r is not None:
        reader = conn_r.cursor()
        reader.execute("SELECT Title, Body, Tags FROM no_dup_train ORDER BY RANDOM() LIMIT 1000000;")
        
if os.path.isfile(write_db):
    conn_w = create_connection(write_db)
    if conn_w is not None:
        tables = checkTableExists(conn_w)
        writer = conn_w.cursor()
        if tables != 0:
            writer.execute("DELETE FROM QuestionsProcessed WHERE 1;")
            print("Cleared all the rows")
        
print("Time taken to run this cell: ", dt.datetime.now() - start)

In [ ]:
#dont run this cell
#to check what the sql statement output is
start = dt.datetime.now()
write_db = "Processed.db"
if os.path.isfile(write_db):
    conn_w = sqlite3.connect(write_db)
    output = pd.read_sql_query("SELECT * FROM QuestionsProcessed WHERE 1", conn_w)
    conn_w.close()
            
        #print("Oh this is the ouput")
print("Time taken to run this cell: ", dt.datetime.now() - start)

In [ ]:
output

In [ ]:
type(reader)

In [ ]:
for row in reader:
    print(row)

In [ ]:
for row in reader:
    print(row[0][:5])

In [ ]:
for row in reader:
    print(row[1])

In [ ]:
for row in reader:
    print(row[1][2])

In [ ]:
dir(reader)

In [ ]:
type(reader)

__ we create a new data base to store the sampled and preprocessed questions __

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
#http://www.bernzilla.com/2008/05/13/selecting-a-random-row-from-an-sqlite-table/
#https://pynative.com/python-cursor-fetchall-fetchmany-fetchone-to-read-rows-from-table/

start = dt.datetime.now()
preprocessed_data_list = []
reader.fetchone()
questions_with_code = 0
len_pre = 0
len_post = 0
questions_processed = 0

for row in reader:
    is_code = 0
    title, question, tags = row[0], row[1], row[2]
    
    if "<code>" in question:
        is_code = 1
        questions_with_code += 1
    x=len(question)+len(title)
    len_pre += x
    
    code = str(re.findall(r'<code>(.*?)</code>', question, flags = re.DOTALL))
    question = re.sub("<code>(.*?)</code>", "", question, flags = re.MULTILINE|re.DOTALL)
    question = striphtml(question.encode('utf-8'))
    title = title.encode('utf-8')
    question = str(title)+" "+str(question)
    question = re.sub(r'[^A-Za-z]+',' ',question)
    words = word_tokenize(str(question.lower()))
    
    #Removing all single letter and and stopwords from question except for the letter 'c'
    question = ' '.join(str(stemmer.stem(j)) for j in words if j not in stop_words and (len(j)!=1 or j=='c'))
    len_post+=len(question)
    tup = (question, code, tags, x, len(question), is_code)
    questions_processed+=1
    writer.execute("insert into QuestionsProcessed(question, code, tags, words_pre, words_post,is_code) values(?,?,?,?,?,?)",tup)
    if(questions_processed%100000==0):
        print("number of questions completed=",questions_processed)
        
no_dup_avg_len_pre = (len_pre*1.0)/questions_processed
no_dup_avg_len_post = (len_post*1.0)/questions_processed

print("Average length of questions(Title+Body) before processing: %d"%no_dup_avg_len_pre)
print("Average length of questions(Title+Body) after processing: %d"%no_dup_avg_len_post)
print("Percentage of questions containing code: %d"%((questions_with_code*100.0)/questions_processed))
print("Time taken to run this cell: ", dt.datetime.now() - start)

In [ ]:
# dont forget to close the connections, or else you will end up with locks
conn_r.commit()
conn_w.commit()
conn_r.close()
conn_w.close()

In [ ]:
if os.path.isfile(write_db):
    conn_r = create_connection(write_db)
    if conn_r is not None:
        reader =conn_r.cursor()
        reader.execute("SELECT question From QuestionsProcessed LIMIT 10")
        print("Questions after preprocessed")
        print('='*100)
        reader.fetchone()
        for row in reader:
            print(row)
            print('-'*100)
conn_r.commit()
conn_r.close()

In [ ]:
start = dt.datetime.now()
write_db = 'Processed.db'
if os.path.isfile(write_db):
    conn_r = create_connection(write_db)
    if conn_r is not None:
        data_in_table = pd.read_sql_query("""SELECT * FROM QuestionsProcessed LIMIT 10""", conn_r)
conn_r.close()
print("Time taken to run this cell: ", dt.datetime.now() - start)

In [ ]:
data_in_table.head()

In [ ]:
#Taking 1 Million entries to a dataframe.
start = dt.datetime.now()
write_db = 'Processed.db'
if os.path.isfile(write_db):
    conn_r = create_connection(write_db)
    if conn_r is not None:
        preprocessed_data = pd.read_sql_query("""SELECT question, Tags FROM QuestionsProcessed""", conn_r)
conn_r.commit()
conn_r.close()
print("Time taken to run this cell is: ",dt.datetime.now()-start)

In [ ]:
preprocessed_data.head()

In [ ]:
print("number of data points in sample :", preprocessed_data.shape[0])
print("number of dimensions :", preprocessed_data.shape[1])